In [1]:
import numpy as np
from numpy.linalg import inv
from scipy import stats
import math
from sys import stdout
from mcc import mcc
import lr

In [2]:
root = '../../bosch/'
root = '../../Desktop/bosch/'

In [3]:
# https://www.kaggle.com/c/bosch-production-line-performance

In [4]:
"""

1,183,747 training observations
1,183,748 test observations

Response is 1: 6,879 times out of 1,183,748

Excluding Id (and Response in numeric):
 2,140 categorical features (like L1_S25_F2117)
   968 numeric features (like L1_S25_F2117)
 1,156 date features (like L1_S25_D2118)

L1_S25_D2118 in date is paired with L1_S25_F2117 (if L1_S25_F2117 is present)
Four lines -- 0 to 3
Lines have different number of sections.

Fit each group in training.
For each x in test, find group for x, get model and predict. If group not found, predict 0.

"""

i = 1

In [5]:
def run(filename):
    groups = {}

    with open(root + filename) as f:
        n = 0
        while True:
            line = f.readline()
            if line == '': # or n == 10000:
                break

            cols = line.strip('\n').split(',')

            if n == 0:
                width = len(cols)
            elif len(cols) == width:
                pattern = ','.join([str(i) for i, x in enumerate(cols) if x != ''])
                group = groups.get(pattern, None)
                if group == None:
                    groups[pattern] = group = []
                group.append(n)

            n += 1
    return groups

In [6]:
groups = run('train_numeric.csv')

In [7]:
filename = 'train_numeric.csv'
N = 1183747

with open(root + filename) as f:
    line = f.readline()
    cols = line.strip('\n').split(',')
    
Z = np.zeros((N, len(cols)), dtype = float)

In [8]:
with open(root + filename) as f:
    n = 0
    line = f.readline()
    while True:
        line = f.readline()
        if line == '':
            break

        Z[n, :] = [np.nan if x == '' else float(x) for x in line.strip('\n').split(',')]
        n += 1

In [ ]:
models = {}
keep = {}
count = 0
bad = 0

clf = lr

for pattern, group in groups.iteritems():
    rows = np.array(group) - 1

    # if len(rows) < 11600:
    #    continue

    cols = np.array([int(x) for x in pattern.split(',')])[1 : -1]        
    y = Z[rows, -1]
    X = Z[rows, :][:, cols]

    indices = []
    for j in range(X.shape[1]):
        if np.var(X[:, j]) > 0.1:
            indices.append(j)

    keep[pattern] = indices
    X = X[:, indices]

    N, D = X.shape
    if N < D or N < 10 or D < 1:
        continue

    try:
        models[pattern] = clf.fit(X, y)
    except:
        bad += 1
    # print mcc(y, predict(models[pattern], X))
    count += 1
    stdout.write('\r%s/%s %s bad' % (count, len(groups), bad))
    stdout.flush()        

1407/16710 873 bad

In [ ]:
def y_hat(i):
    pattern = ','.join([str(i) for i, x in enumerate(Z[i, :]) if x != ''])
    cols = np.array([int(x) for x in pattern.split(',')])[1 : -1]        
    x = Z[i, cols]
    return clf.predict(x[models[pattern], keep['pattern']])
        
y = Z[:, -1]
y_hat = [y_hat(i) for i in range(len(Z))]
print mcc(y, y_hat)